In [ ]:
import os
import psycopg2
import logging
import boto3
import json
from botocore.exceptions import ClientError
import requests
from rawg.db import get_conn, get_last_added, insert_games
from rawg.rawg_api import fetch_new_games
from rawg.storage import save_local, upload_s3
from dotenv import load_dotenv

load_dotenv()
logger = logging.getLogger(__name__)
RAWG_API_KEY = os.getenv('RAWG_API_KEY')
logger.setLevel(logging.INFO)


def get_conn():
    try:
        conn = psycopg2.connect(
            host=os.getenv('DB_HOST'),
            dbname=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASS'),
            port=os.getenv('DB_PORT', 5432)
        )
        conn.autocommit = False
        return conn
    except psycopg2.OperationalError:
        logger.exception("DB connection failed")
        raise

def get_last_updated(conn):
    with conn.cursor() as cur:
        cur.execute("SELECT COALESCE(MAX(updated), 0) FROM rawg_games;")
        return cur.fetchone()[0] or 0


def fetch_new_games(last_updated, page_size=40):
    page = 1
    new_games = []
    while True:
        try:
            resp = requests.get(
                'https://api.rawg.io/api/games',
                params={
                    'key': RAWG_API_KEY,
                    'ordering': '-updated',
                    'page_size': page_size,
                    'page': page
                },
                timeout=10
            )
            logger.info("Connection to RAWG API successful")
            resp.raise_for_status()
        except requests.RequestException:
            logger.exception("RAWG API error")
            break

        data = resp.json()
        filtered = [g for g in data.get('results', []) if g.get('updated', 0) > last_updated]
        if not filtered:
            break

        new_games.extend(filtered)
        if not data.get('next'):
            break
        page += 1

    return new_games
# Este lo he puesto porque quiero probarlo porque no se si lo guarda en una memoria temporal automaticamente
# añadir control de errores y logging
def save_local(games, output_dir="output"):
    """Guarda los juegos nuevos en un archivo JSON en S3."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    file_path = os.path.join(output_dir, f"juegos_nuevos_{len(games)}.json")
    
    with open(file_path, mode='w', encoding='utf-8') as file:
        json.dump(games, file, indent=4, ensure_ascii=False)
    
    return file_path

def upload_s3(file_path, bucket_name="data-rawg", s3_folder="juegos/"):
    """Sube un archivo a un bucket de S3 y registra el evento."""
    s3_client = boto3.client('s3')
    try:
        file_name = os.path.basename(file_path)
        s3_path = os.path.join(s3_folder, file_name)
        
        s3_client.upload_file(file_path, bucket_name, s3_path)
        
        logger.info(f"Archivo '{file_name}' subido exitosamente a S3://{bucket_name}/{s3_path}")
    except ClientError as e:
        logger.error(f"Error al subir el archivo '{file_name}' a S3: {e}")
        raise

def lambda_handler(event, context):
    conn = get_conn()
    try:
        last = get_last_updated(conn)
        logger.info("Last updated in DB: %s", last)

        new = fetch_new_games(last)
        if not new:
            logger.info("No new games.")
            return {'new_count': 0}

        path = save_local(new)
        upload_s3(path)

        count = insert_games(conn, new)
        logger.info("Inserted %d new games", count)
        return {'new_count': count}

    except Exception:
        logger.exception("Handler failed")
        return {'error': True}
    finally:
        conn.close()
